# SPARQL Workshop - Section 05: Advanced SPARQL Query Patterns

In this section, we show you some more advanced SPARQL queries. There are almost an infintesimal number of ways you can write SPARQL patterns to build a queries.  In this next notebook, we want to convey some of the more common query patterns and use cases.  In the process, we will introduce a few of the remaining SPARQL commands.

## Graph Model

As we have in previous notebooks, we've included here a copy of the graph data model for this dataset as reference to aid in writing your queries.

<img src="https://ee-assets-prod-us-east-1.s3.amazonaws.com/modules/f3f89ef4607743429fb01ae23d983197/v1/workshop/images/datamodel-rdf-updated.png"/>


### 05.01 - Recommendation query

One of the more popular use cases for a graph database is for creating recommendations.  You may recall one of our earlier queries that was used to find Friends-of-Friends, or Co-Stars-of-Co-Stars.  Building recommendations can be very similar to that.

Start with a person with ID 'person378' and suggest 10 movie recommendations.
The movies should be highly rated and belong to the favourite genre of this person.

In [ ]:
%%sparql

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>
        
SELECT DISTINCT ?title ?rating ?votes 
WHERE 
{
    { 
        SELECT ?genre (COUNT(?genre) as ?genrecount) 
        WHERE 
        {
            resource:person378 objProp:rated ?movie1 .
            ?movie1 rdf:type class:Movie .
            ?movie1 objProp:genre ?genre .
        } GROUP BY ?genre ORDER BY DESC(?genrecount) LIMIT 1
    }
    
    resource:person378 objProp:rated ?movie1 .
    ?movie1 rdf:type class:Movie .
    ?movie1 objProp:genre ?genre .
    ?movie2 objProp:genre ?genre .
    ?movie2 rdf:type class:Movie .
    ?movie2 dataProp:averageRating ?rating .
    ?movie2 dataProp:numVotes ?votes .
    ?movie2 dataProp:title ?title .
    FILTER (?rating > 8.0 && ?votes > 50000 && ?movie1 != ?movie2)
    
} ORDER BY DESC(?rating) LIMIT 10

### 05.02 - The "Six Degrees of Kevin Bacon"

It wouldn't be a complete graph workshop if we didn't include the "Six Degrees of Kevin Bacon"! :)  If you're not familar with the concept, the rule of ["six degrees of separation"](https://en.wikipedia.org/wiki/Six_degrees_of_separation) postulates that everyone on the planet is connected through no more than six social relationships with anyone else on the planet.  That same concept has been extended within the realm of film, suggesting that all actors/actresses are no more than six degrees (across common co-stars and productions) connected to the actor Kevin Bacon.  (It is also a humorous joke because it seems like Kevin Bacon has performed in A LOT of movies!)

This query tends to be easier to execute in the Property Graph data model.  Property Graph query languages, such as Gremlin - also supported by Neptune, contain commands that more easily allow for recursion with a given query.  With SPARQL, you would need to mimic this type of behavior by using `UNION` between multiple `SELECT` statements.  We'll provide an example here.

In the following query, let's see how many hops there are between James Earl Jones and Kevin Bacon:

In [ ]:
%%sparql

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>
        
SELECT ?title1 ?name2 ?title2 ?name3 WHERE {
    {
        ?artist1 dataProp:name "James Earl Jones" .
        ?movie1 objProp:actor|objProp:actress ?artist1 .
        ?movie1 objProp:actor|objProp:actress ?artist2 .
        ?movie1 dataProp:title ?title1 .
        ?artist2 dataProp:name ?name2 
        FILTER (?name2 = "Kevin Bacon")
    } UNION {
        ?artist1 dataProp:name "James Earl Jones" .
        ?movie1 objProp:actor|objProp:actress ?artist1 .
        ?movie1 objProp:actor|objProp:actress ?artist2 .
        ?movie1 dataProp:title ?title1 .
        ?artist2 dataProp:name ?name2 .
        ?movie2 objProp:actor|objProp:actress ?artist2 .
        ?movie2 objProp:actor|objProp:actress ?artist3 .
        ?movie2 dataProp:title ?title2 .
        ?artist3 dataProp:name ?name3 
        FILTER (?name3 = "Kevin Bacon")
    }
} LIMIT 10

Let's breakdown this query in a bit more detail.  We start with finding the resource URI for James Earl Jones:

`?artist1 dataProp:name "James Earl Jones" .`

We then use a series of triples patterns to find all movies where James Earl Jones appeared and all co-stars of James Earl Jones in those movies:

```
?movie1 objProp:actor|objProp:actress ?artist1 .
?movie1 objProp:actor|objProp:actress ?artist2 .
?movie1 dataProp:title ?title1 .
?artist2 dataProp:name ?name2 .
```

Unfortuately, there is no recursion support within the SPARQL language.  So if at this point we don't find Kevin Bacon in the list of names (`name2`), then we would need to repeat these same statements and go an additional hop in the graph.  We can do all of this in one SPARQL query by chaining these via the `UNION` statement.  Thankfully, there are plenty of two-hop relationships between Kevin Bacon and James Earl Jones, so we need not go any further here.  However, if this query returned an empty result, we may have to add a second `UNION` and extend this even further.

Last, but not least, we want to limit our results.  There are probably thousands of paths between James Earl Jones and Kevin Bacon.  In this case, we only care that a path exist and we may be interested in a few other possible paths.  We can limit the execution time of our query by only returning the first 10 results found.

_NOTE: Neptune executes traversals in a Breadth First execution pattern, by default.  So in many cases (but not all) we will see shorter paths returned first.  You can alter this behavior using [Neptune Query Hints](https://docs.aws.amazon.com/neptune/latest/userguide/sparql-query-hints.html) to perform a Depth First traversal._

### 05.03 - Running average

SPARQL also provides a few aggregation functions that can be applied to the values over aggregated groups. This includes things like `AVG`, `MAX`, and `MIN`. 

Consult the following documentation:
 - [`AVG`](https://www.w3.org/TR/sparql11-query/#defn_aggAvg)
 - [`MAX`](https://www.w3.org/TR/sparql11-query/#defn_aggMax)
 - [`MIN`](https://www.w3.org/TR/sparql11-query/#defn_aggMin)
 
Let's try using the `AVG` function to take a running average of ratings for a particular Artist or production.  In the following two examples we look at the average rating "The Original Kings of Comedy".  In addition, we have a query looking at the average ratings of all movies directed by Roland Emmerich.

In [ ]:
%%sparql

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>
        
SELECT (AVG(?rating) as ?avgrating) 
WHERE 
{
    ?movie rdf:type class:Movie .
    ?movie dataProp:title "The Original Kings of Comedy" .
    
    GRAPH ?redge 
    {
        ?person objProp:rated ?movie .
    }
    
    ?redge dataProp:rating ?rating .
}

In [ ]:
%%sparql

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>
        
SELECT (AVG(?rating) as ?avgrating) 
WHERE 
{
    ?artist rdf:type class:Artist .
    ?artist dataProp:name "Roland Emmerich" .
    ?movie objProp:director ?artist .
    
    GRAPH ?redge 
    {
        ?person objProp:rated ?movie .
    }
    
    ?redge dataProp:rating ?rating .
}

Yikes!  People must really not like Roland Emmerich's films.

### 05.04 - Another GROUP BY / ORDER BY example

Which is the most popular genre of movies that were directed by Martin Scorsese and Leonardo DiCaprio acted in them.


In [ ]:
%%sparql

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>
        
SELECT ?genre (COUNT(?genre) as ?genrecount) 
WHERE 
{
    ?leo rdf:type class:Artist .
    ?leo dataProp:name "Leonardo DiCaprio" .
    ?movies objProp:actor ?leo .
    ?movies objProp:director ?martin .
    ?martin dataProp:name "Martin Scorsese" .
    ?movies objProp:genre ?genre
} 
GROUP BY ?genre 
ORDER BY DESC(?genrecount)

### 05.05 - Projecting Specific Variable Bindings from the Query Result

As mentioned before, we should project out only those variables from the query which we are interested in. Some of the variables introduced in the patterns within the query may be useful only to track the joins between intermiediate results.

Here's a good example. List the top 10 movies which have received highest average rating by the people. Here, we are projecting ?title and ?avg_rating where as there are many more variables within the query.

In [ ]:
%%sparql

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>
        
SELECT ?title ?avg_rating 
WHERE 
{
    ?carrie rdf:type class:Artist .
    ?carrie dataProp:name "Carrie Fisher" .
    ?movies objProp:actress ?carrie .
    ?movies dataProp:title ?title .
    ?movies dataProp:averageRating ?avg_rating .
} 
ORDER BY DESC(?avg_rating) LIMIT 10

### 05.06 - Social Rankings

Lastly, let's look at another example for proving connection based ranking.  In the following example, we will look at users and determine which of them are the most highly connected in our graph.


In [ ]:
%%sparql

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>
        
SELECT ?firstname ?lastname (COUNT(?knows) as ?numknows)
WHERE 
{
    ?person rdf:type class:Person .
    ?person dataProp:firstName ?firstname .
    ?person dataProp:lastName ?lastname .
    ?person objProp:knows ?knows .
} 
GROUP BY ?firstname ?lastname 
ORDER BY DESC(?numknows) 
LIMIT 10

### Review

In the previous notebook we looked at more advanced patterns for using Graph traversals to find interesting insights within our data.  These are just some of the different patterns that make graph useful.  We will use some of these patterns in later exercises in this workshop.

### End of Section 05 - Move onto the next notebook.